In [5]:
from math import pi
import numpy as np
from numpy import arcsin,sin ,arctan2,cos
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import pandas as pd
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
tc = pi/2;
R = 6371
## function to impute missing latitude/longitude 
def latlon(lat1, lon1, d):
    lat1 *= pi/180
    lon1 *= pi/180
    lat =arcsin(sin(lat1)*np.cos(d/R)+cos(lat1)*sin(d)*cos(tc))
    dlon=arctan2(sin(tc)*sin(d/R)*cos(lat1),cos(d)-sin(lat1)*sin(lat))
    lon=np.mod( lon1-dlon+pi,2*pi )-pi
    lat *= 180/pi
    lon *= 180/pi
    lon[lon < 0] = 180 - np.abs(lon[lon < 0])
    lat[lat < 0] = 180 - np.abs(lat[lat < 0])
    return lat,lon



iris=pd.read_csv('intracity_fare_train.csv')
df= pd.DataFrame(iris)

### imputing missing destination coordinates
df.dropna(subset=['STARTING_LATITUDE', 'DESTINATION_LATITUDE'], how='all',inplace=True)

df[['STARTING_LATITUDE','DESTINATION_LATITUDE']] = df[['STARTING_LATITUDE','DESTINATION_LATITUDE']].fillna(value=0)

temp = df['DESTINATION_LATITUDE']==0
df.loc[temp,'DESTINATION_LATITUDE'],df.loc[temp, 'DESTINATION_LONGITUDE'] = latlon(df.loc[temp,'STARTING_LATITUDE'], df.loc[temp,'STARTING_LONGITUDE'], df.loc[temp,'DISTANCE']/2)
temp = df['STARTING_LATITUDE']==0
df.loc[temp,'STARTING_LATITUDE'],df.loc[temp, 'STARTING_LONGITUDE'] = latlon(df.loc[temp,'DESTINATION_LATITUDE'], df.loc[temp,'DESTINATION_LONGITUDE'], df.loc[temp,'DISTANCE']/2)

### Enumerating vehicle types for analysis and training


df['VEHICLE_TYPE']=df['VEHICLE_TYPE'].apply(lambda x: x.upper())

df['TOTAL_LUGGAGE_WEIGHT']=df['TOTAL_LUGGAGE_WEIGHT'].fillna(7) # imputing most accurate value
df['WAIT_TIME']=df['WAIT_TIME'].fillna(15.5) # imputing most accurate value

target_map = {'AC BUS':3,
 'AUTO RICKSHAW':4,
 'BUS':1,
 'METRO':2,
 'MINI BUS':0,
 'TAXI AC':6,
 'TAXI NON AC':5}
df['VEHICLE_TYPE']=df['VEHICLE_TYPE'].apply(lambda x: target_map[x])


### drop duplicates
df.drop_duplicates(subset=[
 'STARTING_LATITUDE',
 'STARTING_LONGITUDE',
 'DESTINATION_LATITUDE',
 'DESTINATION_LONGITUDE',
 'VEHICLE_TYPE',
 'TOTAL_LUGGAGE_WEIGHT',
 'WAIT_TIME',
 'TRAFFIC_STUCK_TIME',
 'DISTANCE',
 'FARE'],inplace=True)
###
###doong same for test data
from sklearn import linear_model
from sklearn.decomposition import PCA
pca = PCA(n_components=8)

tr=df
tE=pd.read_csv('intracity_fare_test.csv')
te=pd.DataFrame(tE)

model=linear_model.LogisticRegression()
tr_X = tr[['STARTING_LATITUDE',
 'STARTING_LONGITUDE',
 'DESTINATION_LATITUDE',
 'DESTINATION_LONGITUDE',
 'VEHICLE_TYPE',
 'TOTAL_LUGGAGE_WEIGHT',
 'WAIT_TIME',
 'TRAFFIC_STUCK_TIME',
 'DISTANCE']]# taking the training data features

tr_y = tr[['FARE']]# output of our training data
#######
te.dropna(subset=['STARTING_LATITUDE', 'DESTINATION_LATITUDE'], how='all',inplace=True)
#print(df.shape)
te[['STARTING_LATITUDE','DESTINATION_LATITUDE']] = te[['STARTING_LATITUDE','DESTINATION_LATITUDE']].fillna(value=0)

temp = te['DESTINATION_LATITUDE']==0
# print(interfare[temp])
te.loc[temp,'DESTINATION_LATITUDE'],te.loc[temp, 'DESTINATION_LONGITUDE'] = latlon(te.loc[temp,'STARTING_LATITUDE'], te.loc[temp,'STARTING_LONGITUDE'], te.loc[temp,'DISTANCE']/2)
temp = te['STARTING_LATITUDE']==0
te.loc[temp,'STARTING_LATITUDE'],te.loc[temp, 'STARTING_LONGITUDE'] = latlon(te.loc[temp,'DESTINATION_LATITUDE'], te.loc[temp,'DESTINATION_LONGITUDE'], te.loc[temp,'DISTANCE']/2)

te['VEHICLE_TYPE']=te['VEHICLE_TYPE'].apply(lambda x: x.upper())
te['TOTAL_LUGGAGE_WEIGHT']=te['TOTAL_LUGGAGE_WEIGHT'].fillna(7)
te['WAIT_TIME']=te['WAIT_TIME'].fillna(0)
target_map = {'AC BUS':3,
 'AUTO RICKSHAW':4,
 'BUS':1,
 'METRO':2,
 'MINI BUS':0,
 'TAXI AC':6,
 'TAXI NON AC':5}

te['VEHICLE_TYPE']=te['VEHICLE_TYPE'].apply(lambda x: target_map[x])

te_X = te[['STARTING_LATITUDE',
 'STARTING_LONGITUDE',
 'DESTINATION_LATITUDE',
 'DESTINATION_LONGITUDE',
 'VEHICLE_TYPE',
 'TOTAL_LUGGAGE_WEIGHT',
 'WAIT_TIME',
 'TRAFFIC_STUCK_TIME',
 'DISTANCE']]# taking the training data features


regr_1 = DecisionTreeRegressor(max_depth=7)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=15),
                              n_estimators=250, random_state=1024)

    #len(prediction)
regr_1.fit(tr_X, tr_y)
regr_2.fit(tr_X, tr_y)
y_1 = regr_1.predict(te_X)
y_2 = regr_2.predict(te_X)
pred=pd.DataFrame(y_2)
pred.index=pred.index+1
# do for 12 and 13
pred.to_csv(path_or_buf='answer_final.csv',header=['FARE'],mode='w',index_label='ID')


C:\Users\hp-\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
